On cherche à calculer l'équivalent carbone de la consommation électrique due aux échanges de données internet. Pour cela, on utilise des informations de flux de données et de présence de personnels du site de Sète, pour se ramener à une consommation de données par jour et par personne. On généralise ensuite cette consommation à l'ensemble du personnel de l'UMR.

On fait un certain nombre d'hypothèses:

- la consommation d'internet par personne par jour n'a pas changé entre 2019 et 2021

- les comportements d'utilisation d'internet du personnel à Sète n'est pas différent de celui des autres sites

- le personnel MARBEC ne travaille que les jours ouvrés


# 1. Calcul quantité de données échangées

On a, par jour, un flux moyen entrant et un flux moyen sortant. Le flux moyen de chaque jour est enregistré le lendemain à 1h du matin

In [1]:
import pandas as pd
import numpy as np
import math
from datetime import timedelta

traffic_df = pd.read_csv("data-donnees-internet/traffic-donnees-sete.csv")
traffic_df['Date']=pd.to_datetime(traffic_df['Sample Date YMD'], infer_datetime_format=True)
traffic_df['Date']=traffic_df['Date']-timedelta(days=1)
traffic_df

,Hostname,Target,Sample Date YMD,Sample Time HHMM,Count in seconds,Incoming traffic in bps (avg day),Outgoing traffic in bps (avg day),Peak inbound traffic in bps,Peak outbound traffic in bps,Date
0,sites,GigabitEthernet1/0,11/23/2020,1:00,1606089600,55192.0,117222.6,601766.4,2346130.7,2020-11-22
1,sites,GigabitEthernet1/0,11/24/2020,1:00,1606176000,5351484.2,1745443.2,46186475.3,21301345.1,2020-11-23
2,sites,GigabitEthernet1/0,11/25/2020,1:00,1606262400,7087875.1,1063973.7,77554496.2,14717147.1,2020-11-24
3,sites,GigabitEthernet1/0,11/26/2020,1:00,1606348800,5238180.8,2139920.4,72566815.4,32529461.6,2020-11-25
4,sites,GigabitEthernet1/0,11/27/2020,1:00,1606435200,6030543.9,5191834.1,35059495.4,70644815.9,2020-11-26
...,...,...,...,...,...,...,...,...,...,...
360,sites,GigabitEthernet1/0,11/18/2021,1:00,1637193600,28440854.6,9821430.1,175365040.4,159933589.3,2021-11-17
361,sites,GigabitEthernet1/0,11/19/2021,1:00,1637280000,27491672.0,6812135.1,175481504.2,88729496.2,2021-11-18
362,sites,GigabitEthernet1/0,11/20/2021,1:00,1637366400,22971880.9,7349417.9,175626957.8,37371580.7,2021-11-19
363,sites,GigabitEthernet1/0,11/21/2021,1:00,1637452800,311307.2,1581631.7,6464377.3,100715441.7,2021-11-20


On passe du flux de données (en bps) à une quantité de données, sur l'ensemble de la journée.

## Quantité de données envoyées

In [2]:
n_hours_in_day = 24
traffic_df['data_out_Gb'] = (3600 * n_hours_in_day * traffic_df['Outgoing traffic in bps (avg day)'] / 10**9)

In [3]:
print('Moyenne journalière de la quantité de données envoyées (Gb): ', traffic_df['data_out_Gb'].mean())

Moyenne journalière de la quantité de données envoyées (Gb):  436.5687608679453


## Quantité de données reçues

In [4]:
traffic_df['data_in_Gb'] = (3600 * n_hours_in_day * traffic_df['Incoming traffic in bps (avg day)'] / 10**9)

In [5]:
print('Moyenne journalière de la quantité de données reçues (Gb): ', traffic_df['data_in_Gb'].mean())

Moyenne journalière de la quantité de données reçues (Gb):  662.7279638347401


# 2. Données de présence sur site

On a accès au nombre de personnes présentes chaque jour sur site

In [6]:
# Lit les données de présence
presence_df = pd.read_csv("data-donnees-internet/presence_sete.csv")

In [7]:
# Renomme les colonnes
presence_df = presence_df.set_axis(["M-D","N"], axis=1)
# Ajoute une colonne année
presence_df = presence_df.assign(Y = np.repeat(str(2021), presence_df.shape[0]))
# Crée un colonne avec la date
presence_df['Date'] = presence_df['M-D'].str.cat(presence_df['Y'], sep='-')
# Met au format date
presence_df['Date'] = pd.to_datetime(presence_df['Date'], infer_datetime_format=True, errors = "coerce")

In [8]:
presence_df

,M-D,N,Y,Date
0,01-04,37.0,2021,2021-01-04
1,01-05,38.0,2021,2021-01-05
2,01-06,32.0,2021,2021-01-06
3,01-07,36.0,2021,2021-01-07
4,01-08,32.0,2021,2021-01-08
...,...,...,...,...
157,12-27,5.0,2021,2021-12-27
158,12-28,4.0,2021,2021-12-28
159,12-29,3.0,2021,2021-12-29
160,12-30,3.0,2021,2021-12-30


# 3. Calcul de la moyenne par jour par personne

On utilise les données de présence sur site et la quantité de données échangée par jour pour déterminer une "consommation" de données internet moyenne **par personne par jour**.

In [9]:
# Merge les deux data frame
df = presence_df.merge(traffic_df)
# Garde uniquement les jours avec plus de 30 personnes sur site
df = df[df.N >= 30]
# Calcul la moyenne par personne
df['data_out_per_p_Gb'] = df['data_out_Gb'] / df['N']
df['data_in_per_p_Gb'] = df['data_in_Gb'] / df['N']

In [10]:
print('Moyenne par jour par personne de la quantité de données envoyées (Gb): ', df['data_out_per_p_Gb'].mean())
print('Ecart type: ', df['data_out_per_p_Gb'].std())

Moyenne par jour par personne de la quantité de données envoyées (Gb):  9.406363959238483
Ecart type:  14.258392734363067


In [11]:
print('Moyenne par jour par personne de la quantité de données reçues (Gb): ', df['data_in_per_p_Gb'].mean())
print('Ecart type: ', df['data_in_per_p_Gb'].std())

Moyenne par jour par personne de la quantité de données reçues (Gb):  21.56091291885494
Ecart type:  24.40654832108056


# 4. Calcul de la consommation totale et équivalent carbone

On utilise le ratio déduit de l'application Carbonalyser, basée sur le "1byte model": https://theshiftproject.org/en/carbonalyser-browser-extension/. On obtient un ratio de 0.35 Wh/Mb = **0.35 kWh/Gb**.

In [12]:
ratio_kWhGb = 0.35
donnees_jour_Gb = df['data_out_per_p_Gb'].mean() + df['data_in_per_p_Gb'].mean()
elec_jour_per_p_kWh = donnees_jour_Gb*ratio_kWhGb

print("Consommation moyenne d'électricité par jour et par personne (kWh): ", elec_jour_per_p_kWh)

Consommation moyenne d'électricité par jour et par personne (kWh):  10.838546907332697


In [13]:
effectif = 265

print("Consommation d'électricité par jour (kWh): ", elec_jour_per_p_kWh*effectif)

Consommation d'électricité par jour (kWh):  2872.2149304431646


In [14]:
n_jours_ouvre = 249

print("Consommation d'électricité totale en 2019 (kWh): ", elec_jour_per_p_kWh*effectif*n_jours_ouvre)

Consommation d'électricité totale en 2019 (kWh):  715181.517680348


In [15]:
ratio_elec_C_fr = 0.112 # en kgCO2/kWh

print("Equivalent C02 (t): ", elec_jour_per_p_kWh*effectif*n_jours_ouvre*ratio_elec_C_fr/10**3)

Equivalent C02 (t):  80.10032998019898
